# "Solow Model"
> "Simple Solow model without technological progress"

- toc:true
- branch: master
- badges: true
- comments: false
- author: João B. Duarte
- categories: [Economic growth, python, dash]


The Solow model is described by the following system of equations:

\begin{align}
Y_t &= A_t F(K_t, L_t) & & \mathbf{\text{Production Function}}\\
\\
\Delta K_t &= I_t - \delta K_t  & & \mathbf{\text{Capital Accumulation}}\\
\\
S_t &= s(Y_t - \delta K_t) & & \mathbf{\text{Net Saving Behaviour}}\\
\\
I_t - \delta K_t &= S_t & & \mathbf{\text{Closed Economy}}
\end{align}


Lets simplify it.

The Closed Economy and Net Saving Behaviour equations imply that net investment is equal to a fixed fraction $s$ of net income:
$I_t - \delta K_t = s(Y_t - \delta K_t)$. We can then replace this into the Capital Accumulation equation to get:

$$\Delta K_t = s(Y_t - \delta K_t)$$

Lets think about this equation for a moment. If there is some net income leftover after paying for restocking the
 capital that gets lost during the latest production ($Y_t - \delta K_t >0$), then we invest part of it into building
 new capital ($\Delta K_t >0$). If, on the other hand, income is not enough to even pay for capital replacement, then
 capital falls next period ($\Delta K_t <0$).

 Finally, lets replace the production function into $Y_t$ to get the equation that summarizes all of the Solow model
 dynamics:

 $$\Delta K_t = s(A_t F(K_t, L_t) - \delta K_t) \quad \quad \text{(In levels)}$$

 And we can rewrite it in growth rates when dividing both sides by $K_t$:

 $$\frac{\Delta K_t}{K_t} = s\frac{A_t F(K_t, L_t)}{K_t} - s\delta \quad \quad \text{(In growth rates)}$$

Hence, so long as output per unit of capital is larger than what gets lost per unit of capital, capital will grow.

So the crucial part to understand capital growth is to figure out how output per unit of capital behaves.

**Assumptions**

1. $$\frac{\partial F(K_t, L_t)}{\partial K_t} > 0 \quad  \frac{\partial^2 F(K_t, L_t)}{\partial K_t^2} < 0 \quad \text{Decreasing returns to scale}$$

2. $$\frac{\Delta L_t}{L_t} = n$$

With assumption (1) in place, as the economy grows, output per unit of capital $\frac{A_t F(K_t, L_t)}{K_t}$ falls.
Which means that at some point it will be lower than depreciation $\delta$. Assumption (2) implies that population growth is constant.

***Solow model set in per capita variable***

Lets also assume a Cobb-Douglas production function: $Y_t = A_t K_t^\alpha L_t^{1-\alpha}$. Note that assumption 1 is verified
under this specific production function.

Let $k_t=\frac{K_t}{L_t}$ and $y_t = \frac{Y_t}{L_t}$ be per capita capital and output, respectively.

Lets use a log approximation $\ln{(1+x)} \approx x$ for small $x$ (the model can also hold exactly, ie. without approximation, but the
algebra does not match the class slides)

Using this approximation, we have that
$$\frac{\Delta k_t}{k_t} \approx \frac{\Delta K_t}{K_t} - \frac{\Delta L_t}{L_t}$$

Why?

$$\ln{k_t} = \ln{K_t} - \ln{L_t}$$

and so

$$\ln{k_{t+1} - \ln{k_t}}  = \ln{K_{t+1}} - \ln{K_t} - (\ln{L_{t+1}} -  \ln{L_t}) $$
$$\ln{\frac{k_{t+1}}{k_t}}  = \ln{\frac{K_{t+1}}{K_t}} - \ln{\frac{L_{t+1}}{L_t}}$$

$$\ln{(1 + \frac{\Delta k_{t}}{k_t})}  = \ln{(1 + \frac{\Delta K_{t}}{K_t})} - \ln{(1+\frac{\Delta L_{t}}{L_t})}$$

Hence,

$$\frac{\Delta k_t}{k_t} \approx \frac{\Delta K_t}{K_t} - \frac{\Delta L_t}{L_t}$$

Now, we have all the ingredients to write the main Solow equation in per capita terms:

$$\frac{\Delta K_t}{K_t} = s\frac{A_t F(K_t, L_t)}{K_t} - s\delta $$

$$\frac{\Delta k_t}{k_t} + \frac{\Delta L_t}{L_t} = s\frac{A_t k_t^\alpha}{k_t} - s\delta $$

 $$ \boxed{\frac{\Delta k_t}{k_t} = s\frac{A_t k_t^\alpha}{k_t} - (s\delta+n)}$$


In [11]:
#hide
import numpy as np
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
JupyterDash.infer_jupyter_proxy_config()
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected"

class Solow:
    r"""
    Implements the Solow growth model with the update rule

        k_{t+1} = [(s z k^α_t) + (1 - δ)k_t] /(1 + n)

    """
    def __init__(self, n=0.05,  # population growth rate
                       s=0.3,  # savings rate
                       δ=0.1,   # depreciation rate
                       α=0.3,   # share of labor
                       z=2.0,   # productivity
                       k=1.0):  # current capital stock

        self.n, self.s, self.δ, self.α, self.z = n, s, δ, α, z
        self.k = k

    def f(self, k):
        return self.z * k**self.α

    def savings(self, k):
        return self.s*self.f(k)/k

    def depreciation(self):
        return self.δ*self.s + self.n
solow = Solow()
solow2 = Solow()
k = np.linspace(1,100,100)
fig = go.Figure()
fig.add_trace(go.Scatter(x=k, y=solow.savings(k), name = 'Savings benchmark',
              line=dict(color='black', width=1)))
fig.add_trace(go.Scatter(x=k, y=solow.depreciation()*np.ones(100), name = 'Depreciation benchmark',
              line=dict(color='black', width=1)))
fig.add_trace(go.Scatter(x=k, y=solow2.savings(k), name = 'Savings',
              line=dict(color='blue', width=2)))
fig.add_trace(go.Scatter(x=k, y=solow2.depreciation()*np.ones(100), name = 'Depreciation',
              line=dict(color='red', width=2)))
fig.update_layout(title='Simulations',
                   xaxis_title='k',
                   yaxis_title='Value')
fig.update_layout(transition_duration=1000)
fig.show()


In [12]:
#hide

# Build App
app = JupyterDash(__name__)
server = app.server
app.layout = html.Div([
    html.H1(
        children='Solow Model',
        style={
            'textAlign': 'center'
        }
    ),
     html.Label('Savings rate'),
    dcc.Slider(
        id='s-slider',
        min=0.1,
        max=0.9,
        step=0.1,
        marks={i: str(round(i,1)) for i in np.linspace(0.1,0.9,9)},
        value=0.3
    ),
    html.Label('Depreciation rate'),
    dcc.Slider(
        id='delta-slider',
        min=0.1,
        max=0.9,
        step=0.1,
        marks={i: str(round(i,1)) for i in np.linspace(0.1,0.9,9)},
        value=0.1
    ),
    html.Label('Population growth rate'),
    dcc.Slider(
        id='n-slider',
        min=0.01,
        max=0.09,
        step=0.01,
        marks={i: str(round(i,2)) for i in np.linspace(0.01,0.09,9)},
        value=0.05
    ),
    html.Label('Alpha'),
    dcc.Slider(
        id='alpha-slider',
        min=0.1,
        max=0.9,
        step=0.1,
        marks={i: str(round(i,1)) for i in np.linspace(0.1,0.9,9)},
        value=0.3
    ),
    html.Label('Technology'),
    dcc.Slider(
        id='tec-slider',
        min=1,
        max=9,
        step=1,
        marks={i: str(round(i,1)) for i in range(1,10)},
        value=2
    ),
    dcc.Graph(id='graph', figure = fig)
])
# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    [Input("s-slider", "value"), Input("delta-slider", "value"),
     Input("n-slider", "value"), Input("alpha-slider", "value"), Input("tec-slider", "value")]
)
def update_figure(selected_s, selected_delta, selected_n, selected_α, selected_z):
    k = np.linspace(1,100,100)
    solow2.s = selected_s
    solow2.δ = selected_delta
    solow2.n = selected_n
    solow2.α = selected_α
    solow2.z = selected_z
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=k, y=solow.savings(k), name = 'Savings benchmark',
                  line=dict(color='black', width=1)))
    fig.add_trace(go.Scatter(x=k, y=solow.depreciation()*np.ones(100), name = 'Depreciation benchmark',
                  line=dict(color='black', width=1)))
    fig.add_trace(go.Scatter(x=k, y=solow2.savings(k), name = 'Savings',
                  line=dict(color='blue', width=2)))
    fig.add_trace(go.Scatter(x=k, y=solow2.depreciation()*np.ones(100), name = 'Depreciation',
                  line=dict(color='red', width=2)))
    fig.update_layout(title='Simulations',
                   xaxis_title='k',
                   yaxis_title='Value')
    fig.update_layout(transition_duration=1000)
    return fig
# Run app and display result inline in the notebook
app.run_server(mode='inline')
